In [1]:
from pymilvus import connections, Collection, utility

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

In [1]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="sk-e4f5d5efbab54ebd9207f3865426edfd",  # 如果您没有配置环境变量，请在此处用您的API Key进行替换
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"  # 百炼服务的base_url
)

completion = client.embeddings.create(
    model="text-embedding-v4",
    input=['风急天高猿啸哀', '渚清沙白鸟飞回', '无边落木萧萧下', '不尽长江滚滚来'],
    dimensions=1024,# 指定向量维度（仅 text-embedding-v3及 text-embedding-v4支持该参数）
    encoding_format="float"
)

print(completion.model_dump_json())

{"data":[{"embedding":[0.04918334633111954,-0.06888676434755325,0.047604065388441086,-0.06778378039598465,-0.009488224983215332,-0.07911449670791626,-0.006082115229219198,0.05700455233454704,-0.061416514217853546,-0.00044808987877331674,-0.04071037471294403,0.04928361997008324,0.02147071249783039,-0.04414468631148338,-0.06873635947704315,-0.07299791276454926,-0.07344913482666016,-0.046651482582092285,0.012089026160538197,0.015542138367891312,0.05209122970700264,0.04238992929458618,0.032889168709516525,0.016945945098996162,-0.08071884512901306,-0.022059809416532516,0.003644255455583334,-0.03795289993286133,-0.031836315989494324,-0.022987324744462967,-0.05103837698698044,0.011142710223793983,-0.033666275441646576,0.01806146837770939,0.01007105503231287,0.0001834074209909886,-0.05061222240328789,0.08342618495225906,-0.04040956124663353,-0.013887652195990086,-0.02534371241927147,-0.0005417030770331621,-0.022761711850762367,-0.05389612540602684,0.009820375591516495,0.006812219507992268,-0.0

In [13]:
len(completion.model_dump()['data'])

4

In [14]:
completion.model_dump()['usage']

{'prompt_tokens': 29, 'total_tokens': 29}

### 与Milvus数据库进行交互

In [19]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://172.25.208.1:19530",
    token="root:Milvus"
)

In [24]:
client.create_database(
    db_name="my_database_1"
)

In [25]:
client.create_database(
    db_name="my_database_2",
    properties={
        "database.replica.number": 3
    }
)

In [26]:
client.list_databases()

['default', 'my_database_1', 'my_database_2']

In [28]:
client.describe_database(
    db_name="my_database_1"
)

{'name': 'my_database_1'}

In [29]:
client.alter_database_properties(
    db_name="my_database_1",
    properties={
        "database.max.collections": 10
    }
)

In [30]:
client.drop_database_properties(
    db_name="my_database_1",
    property_keys=[
        "database.max.collections"
    ]
)

In [34]:
client.use_database(
    db_name="my_database_1"
)

In [33]:
client.drop_database(
    db_name="my_database_2"
)

In [43]:
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

In [44]:
from pymilvus import DataType
schema.add_field(field_name="my_id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="my_vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="my_varchar", datatype=DataType.VARCHAR, max_length=512)

{'auto_id': False, 'description': '', 'fields': [{'name': 'my_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'my_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 5}}, {'name': 'my_varchar', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'enable_dynamic_field': True}

In [45]:
# 删除其中一个field
schema.fields

[{'name': 'my_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False},
 {'name': 'my_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 5}},
 {'name': 'my_varchar', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}]

In [46]:
# 3.3. Prepare index parameters
index_params = client.prepare_index_params()

# 3.4. Add indexes
index_params.add_index(
    field_name="my_id",
    index_type="AUTOINDEX"
)

index_params.add_index(
    field_name="my_vector", 
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

In [47]:
# 3.5. Create a collection with the index loaded simultaneously
client.create_collection(
    collection_name="customized_setup_1",
    schema=schema,
    index_params=index_params
)

res = client.get_load_state(
    collection_name="customized_setup_1"
)

print(res)

# Output
#
# {
#     "state": "<LoadState: Loaded>"
# }

{'state': <LoadState: Loaded>}


In [48]:
client.create_collection(
    collection_name="customized_setup_2",
    schema=schema,
)

res = client.get_load_state(
    collection_name="customized_setup_2"
)

print(res)

{'state': <LoadState: NotLoad>}


In [49]:
res = client.list_collections()

print(res)

['customized_setup_1', 'customized_setup_2']


In [52]:
res = client.describe_collection(
    collection_name="customized_setup_2"
)

res

{'collection_name': 'customized_setup_2',
 'auto_id': False,
 'num_shards': 1,
 'description': '',
 'fields': [{'field_id': 100,
   'name': 'my_id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'is_primary': True},
  {'field_id': 101,
   'name': 'my_vector',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 5}},
  {'field_id': 102,
   'name': 'my_varchar',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 512}}],
 'functions': [],
 'aliases': [],
 'collection_id': 460688656129098971,
 'consistency_level': 2,
 'properties': {},
 'num_partitions': 1,
 'enable_dynamic_field': True,
 'created_timestamp': 460690424716853252,
 'update_timestamp': 460690424716853252}

In [55]:
res = client.list_partitions(
    collection_name="customized_setup_2"
)

print(res)

['_default']
